# 使用Strands Agent访问TiDB实现智能分析

## 1. 安装Python依赖

In [5]:
!pip install -q python-dotenv==1.0.0 strands-agents[openai]==1.0.0 strands-agents>=1.0.0 strands-agents-tools>=0.2.0 pytidb==0.0.9 litellm==1.74.9.post1

## 2. 向量化数据

在之前的初始化数据库时，已经创建了几个表，以及初始化了订单、商品等表的数据，但是对于订单记录和客服记录，需要在插入数据的同时，将文本内容进行向量化，在保存到数据库表中。

将商品评论数据保存到csv文件中：

In [2]:
%%writefile product_reviews.csv
review_id,order_id,product_id,customer_id,rating,product_name,review_content,review_content_vec,review_date,is_verified_purchase
"R001","O001","P001","C001","5","品牌:TechZen 类型:智能手机 商品名:ZenPhone Pro 256GB","手机很棒，拍照效果非常好，系统流畅，值得购买！","","2024-01-20 10:00:00",TRUE
"R002","O006","P004","C006","4","品牌:DigitalWave 类型:平板电脑 商品名:SmartPad Elite","平板屏幕清晰，性能不错，就是价格有点贵","","2024-01-30 14:30:00",TRUE
"R003","O008","P008","C001","5","品牌:ProTech 类型:笔记本电脑 商品名:BusinessLaptop X1","笔记本性能强劲，屏幕显示效果很好，办公游戏都很棒","","2024-02-05 09:15:00",TRUE
"R004","O003","P005","C003","2","品牌:TechZen 类型:蓝牙耳机 商品名:SoundBuds Pro 2","耳机连接不稳定，经常断连，音质也不如宣传的那么好，准备退货","","2024-01-25 16:20:00",TRUE
"R005","O007","P006","C007","1","品牌:AudioMax 类型:蓝牙耳机 商品名:AudioMax WX-2000","耳机质量有问题，用了两天就出现杂音，非常失望","","2024-02-02 11:45:00",TRUE
"R006","O010","P005","C009","5","品牌:TechZen 类型:蓝牙耳机 商品名:SoundBuds Pro 2","SoundBuds音质很好，佩戴舒适，降噪效果不错，非常满意","","2024-02-10 14:20:00",TRUE
"R007","O012","P007","C011","4","品牌:TechZen 类型:笔记本电脑 商品名:PowerBook Pro 14寸","PowerBook性能强大，做设计很流畅，就是发热有点大","","2024-02-15 16:30:00",TRUE
"R008","O014","P006","C013","5","品牌:AudioMax 类型:蓝牙耳机 商品名:AudioMax WX-2000","AudioMax耳机音质一流，降噪效果很棒，强烈推荐！","","2024-02-20 11:15:00",TRUE
"R009","O009","P001","C008","3","品牌:TechZen 类型:智能手机 商品名:ZenPhone Pro 256GB","手机整体不错，但电池续航一般，一天需要充两次电","","2024-02-08 13:45:00",TRUE
"R010","O011","P002","C010","2","品牌:FlashTech 类型:智能手机 商品名:FlashMobile Ultra 512GB","FlashMobile手机系统有时卡顿，相机效果也不如宣传的好","","2024-02-12 10:30:00",TRUE
"R011","O013","P003","C012","4","品牌:TechZen 类型:平板电脑 商品名:TabletMax Pro 12.9寸","TabletMax屏幕质量很好，适合看视频和办公，就是存储空间有点小","","2024-02-17 15:20:00",TRUE
"R012","O015","P008","C014","1","品牌:ProTech 类型:笔记本电脑 商品名:BusinessLaptop X1","笔记本键盘有问题，多个按键失灵，影响正常使用","","2024-02-22 09:40:00",TRUE

Overwriting product_reviews.csv


还有客服记录数据：

In [3]:
%%writefile customer_service_records.csv
record_id,order_id,customer_id,service_type,issue_category,conversation_log,conversation_log_vec,service_agent,created_date,resolved_date,resolution_status,product_name
"CS001","O002","C002","订单取消","客户主动取消","Customer: 你好，我想取消订单O002，不需要这个商品了\nAgent: 您好，我来帮您处理订单取消。请稍等，我查看一下订单状态\nAgent: 您的订单还未发货，可以取消。我现在为您处理取消手续\nCustomer: 好的，谢谢。退款什么时候能到账？\nAgent: 退款会在1-3个工作日内原路返回到您的支付账户",,"客服小王","2024-01-16 15:30:00","2024-01-16 16:00:00","已解决","品牌:TechZen 类型:平板电脑 商品名:TabletMax Pro 12.9寸"
"CS002","O003","C003","退货申请","产品质量问题","Customer: 我买的SoundBuds Pro连接不稳定，经常断连，我要退货\nAgent: 您好，很抱歉给您带来不好的体验。请问具体是什么情况？\nCustomer: 和手机连接后经常自动断开，音质也不好\nAgent: 我理解您的困扰。我们支持7天无理由退货，我来为您办理退货手续\nCustomer: 需要我自己承担运费吗？\nAgent: 由于是产品质量问题，运费由我们承担。我安排快递员上门取件",,"客服小李","2024-01-22 10:15:00","2024-01-23 14:30:00","处理中","品牌:TechZen 类型:蓝牙耳机 商品名:SoundBuds Pro 2"
"CS003","O004","C004","换货申请","产品功能异常","Customer: 我的PowerBook键盘有几个键按不了，才买3天\nAgent: 您好，很抱歉出现这个问题。请问是哪几个按键？\nCustomer: 空格键和回车键经常失灵，影响正常使用\nAgent: 这确实影响使用体验。我为您申请换货，换一台全新的\nCustomer: 换货要多久？我工作急需用\nAgent: 我们会优先处理，明天安排取件，后天新机器就能送到",,"客服小张","2024-01-25 09:20:00","2024-01-26 11:45:00","已解决","品牌:TechZen 类型:笔记本电脑 商品名:PowerBook Pro 14寸"
"CS004","O007","C007","退款申请","产品质量问题","Customer: Sony耳机用了两天就出现杂音，我要退款\nAgent: 您好，请问具体是什么故障？\nCustomer: 左耳机有明显的电流声，右耳机声音很小\nAgent: 我来帮您处理。请问您还保留购买凭证和包装吗？\nCustomer: 都在，发票和盒子都没扔\nAgent: 好的，我现在为您登记退款申请，需要1-2个工作日审核",,"客服小陈","2024-02-03 13:40:00","","处理中","品牌:AudioMax 类型:蓝牙耳机 商品名:AudioMax WX-2000"
"CS005","O005","C005","物流咨询","配送延迟","Customer: 我的手机怎么还没到？已经3天了\nAgent: 您好，我来帮您查询物流信息\nAgent: 您的包裹目前在杭州转运中心，预计明天能送达\nCustomer: 为什么这么慢？\nAgent: 最近快递量比较大，给您造成不便很抱歉",,"客服小刘","2024-01-24 16:10:00","2024-01-24 16:20:00","已解决","品牌:FlashTech 类型:智能手机 商品名:FlashMobile Ultra 512GB"
"CS006","O003","C003","退货跟进","退货流程","Customer: 我的退货处理得怎么样了？\nAgent: 您好，我查看一下退货进度\nAgent: 您的退货商品我们已经收到，正在质检\nCustomer: 什么时候能退款？\nAgent: 质检通过后3-5个工作日内退款会到账",,"客服小王","2024-01-28 14:25:00","2024-01-28 14:40:00","已解决","品牌:TechZen 类型:蓝牙耳机 商品名:SoundBuds Pro 2"
"CS007","O004","C004","换货跟进","物流查询","Customer: 我的换货商品发了吗？\nAgent: 您好，我来查询一下\nAgent: 您的换货商品已经发出，快递单号是YD1111222233\nCustomer: 什么时候能到？\nAgent: 预计明天下午能送达，请保持电话畅通",,"客服小李","2024-01-30 10:30:00","2024-01-30 10:45:00","已解决","品牌:TechZen 类型:笔记本电脑 商品名:PowerBook Pro 14寸"
"CS008","O011","C010","退货申请","产品不满意","Customer: FlashMobile手机系统太卡了，我要退货\nAgent: 您好，请问具体是什么情况？\nCustomer: 打开应用很慢，经常卡死，和宣传不符\nAgent: 我理解您的困扰。请问您试过重启或恢复出厂设置吗？\nCustomer: 试过了，没有改善，我就要退货\nAgent: 好的，我为您办理退货手续，快递员明天上门取件",,"客服小王","2024-02-10 14:20:00","2024-02-10 15:30:00","已解决","品牌:FlashTech 类型:智能手机 商品名:FlashMobile Ultra 512GB"
"CS009","O013","C012","换货申请","屏幕问题","Customer: TabletMax屏幕有亮点，影响使用\nAgent: 您好，请问亮点在什么位置？大小如何？\nCustomer: 屏幕中央有个白点，很明显，看视频很影响\nAgent: 这确实影响使用体验。我为您申请换货，换一台全新的\nCustomer: 需要多久？\nAgent: 2-3个工作日，我们会优先处理",,"客服小张","2024-02-15 10:45:00","2024-02-15 11:20:00","已解决","品牌:TechZen 类型:平板电脑 商品名:TabletMax Pro 12.9寸"
"CS010","O015","C014","退款申请","产品缺陷","Customer: 笔记本键盘多个按键失灵，要求退款\nAgent: 您好，请问是哪些按键失灵？\nCustomer: W、A、S、D这几个按键都按不了，游戏都玩不了\nAgent: 这确实很影响使用。请问您是什么时候购买的？\nCustomer: 上周买的，才用了几天\nAgent: 我为您登记退款申请，由于是质量问题，全额退款",,"客服小陈","2024-02-20 16:30:00","","处理中","品牌:ProTech 类型:笔记本电脑 商品名:BusinessLaptop X1"
"CS011","O009","C008","物流咨询","配送延迟","Customer: 我的iPhone为什么还没到？\nAgent: 您好，我来帮您查询物流信息\nAgent: 您的包裹目前在西安转运中心，预计今天晚上能送达\nCustomer: 好的，谢谢",,"客服小刘","2024-02-06 14:15:00","2024-02-06 14:25:00","已解决","品牌:TechZen 类型:智能手机 商品名:ZenPhone Pro 256GB"
"CS012","O016","C015","产品咨询","使用问题","Customer: 华为平板怎么连接蓝牙耳机？\nAgent: 您好，我来教您连接步骤\nAgent: 请打开设置-蓝牙，确保蓝牙开启，然后点击搜索设备\nCustomer: 找到了，已经连接成功，谢谢\nAgent: 不客气，还有其他问题随时联系我们",,"客服小李","2024-02-22 11:20:00","2024-02-22 11:35:00","已解决","品牌:DigitalWave 类型:平板电脑 商品名:SmartPad Elite"
"CS013","O010","C009","售后咨询","保修问题","Customer: AirPods保修多久？如果坏了怎么办？\nAgent: 您好，AirPods享受1年全球联保\nAgent: 如果出现非人为损坏，可以免费维修或更换\nCustomer: 那人为损坏呢？\nAgent: 人为损坏需要支付维修费用，具体价格根据损坏程度而定",,"客服小王","2024-02-08 15:40:00","2024-02-08 15:55:00","已解决","品牌:TechZen 类型:蓝牙耳机 商品名:SoundBuds Pro 2"

Writing customer_service_records.csv


初始化环境变量,这里需要根据创建的TiDB数据库的连接信息，以及硅基流动上的key来更新下面的内容：

In [4]:
import os

from pytidb import TiDBClient
from pytidb.schema import TableModel, Field
from pytidb.embeddings import EmbeddingFunction
from pytidb.datatype import Text

import pandas as pd

os.environ["TIDB_HOST"] = "{host}"
os.environ["TIDB_PORT"] = "4000"
os.environ["TIDB_USERNAME"] = "{username}"
os.environ["TIDB_PASSWORD"] = "{password}"
os.environ["TIDB_DATABASE"] = "{database}"
os.environ["SILICONFLOW_KEY"] = "{key}"
os.environ["AWS_REGION_NAME"] = "us-east-1"

创建TiBD数据库连接的对象：

In [7]:
db = TiDBClient.connect(
    host=os.getenv("TIDB_HOST"),
    port=int(os.getenv("TIDB_PORT", "4000")),
    username=os.getenv("TIDB_USERNAME"),
    password=os.getenv("TIDB_PASSWORD"),
    database=os.getenv("TIDB_DATABASE"),
    enable_ssl=True,
)

创建向量化模型，用于之后的将文本数据向量化：

In [8]:
text_embed = EmbeddingFunction(
    "bedrock/amazon.titan-embed-text-v2:0",
    dimensions = 1024,
    timeout = 60
)

下面，定义两个表的Python表结构，然后读取csv文件，插入表中：

In [9]:
# Define the table
table_name = "product_reviews"
class ProductReview(TableModel, table=True):
    __tablename__ = table_name
    __table_args__ = {"extend_existing": True}

    review_id: str = Field(primary_key=True)
    order_id: str = Field()
    product_id: str = Field()
    customer_id: str = Field()
    rating: int = Field()
    product_name: str = Field()
    # product_name: str = FullTextField()
    review_content: str = Field(sa_type=Text)
    review_content_vec: list[float] = text_embed.VectorField(
        source_field="review_content"
    )
    review_date: str = Field()
    is_verified_purchase: bool = Field()
    

cs_table_name = "customer_service_records"
class CSRecords(TableModel, table=True):
    __tablename__ = cs_table_name
    __table_args__ = {"extend_existing": True}

    record_id: str = Field(primary_key=True)
    order_id: str = Field()
    customer_id: str = Field()
    service_type: str = Field()
    issue_category: str = Field()
    service_agent: str = Field()
    conversation_log: str = Field(sa_type=Text)
    conversation_log_vec: list[float] = text_embed.VectorField(
        source_field="conversation_log"
    )
    created_date: str = Field()
    resolved_date: str = Field()
    resolution_status: str = Field()
    product_name: str = Field()

在进行向量化时，我们只需要插入数据，（记录中向量的字段必须为空），这样，TiDB的Python库就会自动生成向量并保存到该字段中。

In [10]:
pr_table = db.open_table(table_name)
cs_table = db.open_table(cs_table_name)

if pr_table is None:
    pr_table = db.create_table(schema=ProductReview, mode="exist_ok")
if cs_table is None:
    cs_table = db.create_table(schema=CSRecords, mode="exist_ok")

# Read data from CSV file
if pr_table.rows() == 0:
    print("Inserting product_reviews data into table...")

    df = pd.read_csv('product_reviews.csv').fillna('')
    reviews_data = df.to_dict('records')

    # Insert records into table
    data_list = []
    for review in reviews_data:
        del review['review_content_vec'] # 需要删掉这个值，否则不会自动生成
        data_list.append(ProductReview(**review))
    pr_table.bulk_insert(data_list)
    print("Products reviews data inserted.")

if cs_table.rows() == 0:
    print("Inserting customer_service_records data into table...")

    df2 = pd.read_csv('customer_service_records.csv').fillna('')
    cs_data = df2.to_dict('records')

    # Insert records into table
    cs_list = []
    for cs in cs_data:
        del cs['conversation_log_vec'] # 需要删掉这个值，否则不会自动生成
        cs_list.append(CSRecords(**cs))
    cs_table.bulk_insert(cs_list)
    print("customer service records data inserted.")

最后，进行查询验证：

In [11]:
list1 = pr_table.search("ZenPhone").limit(3).to_list()
print("list1:", list1)

list2 = cs_table.search("ZenPhone").limit(3).to_list()
print("list2:", list2)

list1: [{'review_id': 'R001', 'order_id': 'O001', 'product_id': 'P001', 'customer_id': 'C001', 'rating': 5, 'product_name': '品牌:TechZen 类型:智能手机 商品名:ZenPhone Pro 256GB', 'review_content': '手机很棒，拍照效果非常好，系统流畅，值得购买！', 'review_content_vec': array([-0.01210085,  0.02602298, -0.01609586, ..., -0.03662402,
       -0.0375113 , -0.00281314], dtype=float32), 'review_date': '2024-01-20 10:00:00', 'is_verified_purchase': True, '_distance': 0.7635366884382245, '_score': 0.23646331156177547}, {'review_id': 'R009', 'order_id': 'O009', 'product_id': 'P001', 'customer_id': 'C008', 'rating': 3, 'product_name': '品牌:TechZen 类型:智能手机 商品名:ZenPhone Pro 256GB', 'review_content': '手机整体不错，但电池续航一般，一天需要充两次电', 'review_content_vec': array([-0.03290067,  0.04844997, -0.03593287, ..., -0.01771899,
       -0.00185138,  0.01768655], dtype=float32), 'review_date': '2024-02-08 13:45:00', 'is_verified_purchase': True, '_distance': 0.8658629390491379, '_score': 0.13413706095086209}, {'review_id': 'R008', 'order_id': 'O014', 

## 使用TiDB MCP tool创建Agent

下面，我们创建Strands Agent，使用TiDB提供的MCP tools来作为Agent的tool：

In [72]:
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient

stdio_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="python", 
        args=["-m", 'pytidb.ext.mcp'],
        env={
            "TIDB_HOST": os.getenv("TIDB_HOST"),
            "TIDB_PORT": os.getenv("TIDB_PORT"),
            "TIDB_USERNAME": os.getenv("TIDB_USERNAME"),
            "TIDB_PASSWORD": os.getenv("TIDB_PASSWORD"),
            "TIDB_DATABASE": os.getenv("TIDB_DATABASE"),
            "SILICONFLOW_KEY": os.getenv("SILICONFLOW_KEY"),
            "AWS_REGION_NAME": os.getenv("AWS_REGION_NAME")
        }
    )
))

# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

打印tools，保证MCP创建成功：

In [73]:
tools

下面，创建Strands Agent，我们先定义模型，可以根据需要使用Deepseek V3的模型，或者nova模型。

In [45]:
from strands.models import BedrockModel
from strands.models.openai import OpenAIModel
import boto3

ds_model = OpenAIModel(
    client_args = {
        "api_key": os.getenv("SILICONFLOW_KEY"),
        "base_url": "https://api.siliconflow.cn/v1"
    },
    model_id = "deepseek-ai/DeepSeek-V3",
)
bedrock_model = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    boto_session=boto3.Session(region_name="us-east-1")
)


创建Strands Agent：

In [46]:
agent = Agent(
    model=ds_model,
    system_prompt="你是一个客服助手！如果需要回答客服、订单、商品等数据相关问题，可以查询TiDB来回答。",
    tools=tools,
)

设置日志级别：

In [61]:
import logging

# logging.getLogger("strands").setLevel(logging.DEBUG) # 如果想查看所有的中间结果，这用这个设置
logging.getLogger("strands").setLevel(logging.WARNING) # 默认都只查看警告级别的日志

logging.getLogger("strands.models").setLevel(logging.WARNING)
logging.getLogger("strands.event_loop.event_loop").setLevel(logging.DEBUG) # 这个设置，会打印每次所使用的工具

In [54]:
with stdio_mcp_client:
    agent("有哪些退货的订单？")

DEBUG | strands.models.openai | formatting request
DEBUG | strands.models.openai | formatted request=<{'messages': [{'role': 'system', 'content': '你是一个客服助手！如果需要回答客服、订单、商品等数据相关问题，可以查询TiDB来回答。'}, {'role': 'user', 'content': [{'text': '有哪些退货的订单？', 'type': 'text'}]}, {'role': 'assistant', 'content': [], 'tool_calls': [{'function': {'arguments': '{}', 'name': 'show_tables'}, 'id': '019875d948f03e0b9dbeea79a6396849', 'type': 'function'}]}, {'role': 'tool', 'tool_call_id': '019875d948f03e0b9dbeea79a6396849', 'content': [{'text': 'customer_service_records', 'type': 'text'}, {'text': 'logistics', 'type': 'text'}, {'text': 'orders', 'type': 'text'}, {'text': 'product_reviews', 'type': 'text'}, {'text': 'products', 'type': 'text'}]}, {'role': 'assistant', 'content': [], 'tool_calls': [{'function': {'arguments': '{"sql_stmt": "SELECT * FROM orders WHERE status = \'returned\' LIMIT 100;"}', 'name': 'db_query'}, 'id': '019875d9572c260217308cdd22f9432a', 'type': 'function'}]}, {'role': 'tool', 'tool_

目前，通过查询 `orders` 表的 `order_status` 字段，未找到明确标记为“退货”（如 `returned`、`refund` 等）的订单记录。

### 可能原因：
1. **系统中暂无退货订单**。
2. **退货状态存储在其他字段或其他表**（如 `customer_service_records`）。
3. **退货状态使用了其他关键词**（如“已退”、“退款中”等）。

### 下一步建议：
1. **检查其他表**：比如 `customer_service_records` 可能存储了退货或退款记录。
2. **调整查询关键词**：比如尝试模糊匹配退货相关词汇。
3. **确认字段名称**：部分系统中退货状态可能不叫 `order_status`，而是 `refund_status` 或其他字段。

如果需要，我可以帮助查询其他表，或者调整查询条件进一步检查。请确认您的需求！

DEBUG | strands.models.openai | finished streaming response from model


由于这个Agent比较简单，没有使用memory，如果想体验更好的显示效果以及多轮对话，请使用Python应用的方式运行。

## 创建带语义搜索功能的Agent

TiDB提供的MCP tool没有语义搜索的功能，我们使用Strands SDK创建一个能够进行语义搜索的工具：

In [55]:
from strands import tool

@tool
def universal_search(table_name, query, filter_object, limit) -> list[dict]:
    """
    Execute universal search on TiDB database, including semantic search with vector search,
    and filtering some columns.

    Args:
    - table_name: the name of the table to query, should be one of the defined tables like product_reviews or customer_service_records
    - query: query sentance for semantic search
    - filter_object: for filtering one or serval columns, this param is an object with syntax:
    ```{
        "$and|$or": [
            {
                "field_name": {
                    <operator>: <value>
                }
            },
            {
                "field_name": {
                    <operator>: <value>
                }
            }
            ...
        ]
    }```
    or just simple filter like:
    ```{
        "field_name": {
            <operator>: <value>
        }
    }```
    operators are:
        $eq 	Equal to value
        $ne 	Not equal to value
        $gt 	Greater than value
        $gte 	Greater than or equal to value
        $lt 	Less than value
        $lte 	Less than or equal to value
        $in 	In a list of values
    this is an example:
    ```
        {
            "$and": [
                {
                    "created_at": {
                        "$gt": "2024-01-01"
                    }
                },
                {
                    "meta.category": {
                        "$in": ["tech", "science"]
                    }
                }
            ]
        }
    ```
    - limit: for limiting the number of results
    """
    try:
        if table_name == "product_reviews":
            vec_table = db.create_table(schema=ProductReview, mode="exist_ok")
        elif table_name == "customer_service_records":
            vec_table = db.create_table(schema=CSRecords, mode="exist_ok")
        else:
            log.error(f"Invalid table name: {table_name}")
            raise ValueError(f"Invalid table name: {table_name}")

        if type(filter_object) is str:
            filter_object = json.loads(filter_object)
        data_list = vec_table.search(query).filter(filter_object).limit(limit).to_list()
        return data_list
    except Exception as e:
        log.error(f"Failed to execute query: {query}, on table: {table_name}, to filter: {filter_object}, limit: {limit}, error: {e}")
        raise e

这个工具中，定义了详细的描述信息，用于指导模型何时使用该工具，如果要使用，参数是什么、参数格式等信息。


下面再定一个另一个Agent，有语义搜索的功能：

In [59]:
agent_ext = Agent(
    model=ds_model,
    system_prompt="""你是一个客服助手！
    如果需要回答客服、订单、商品等数据相关问题，可以查询TiDB来回答。
    查询TiDB中的数据时，对于vector类型的记录，不需要在结果中返回该字段，而且可以针对有vector类型字段的表进行语义搜索。
    """,
    callback_handler=None,
    tools=tools + [universal_search],
)

In [65]:
with stdio_mcp_client:
    res = agent_ext("从TiDB数据库中，查找商品P005,有关蓝牙连接问题的商品评论信息")

DEBUG | strands.event_loop.event_loop | tool_use=<{'toolUseId': '019875e99106f61ec53aa88c217947fa', 'name': 'db_query', 'input': {'sql_stmt': "SELECT * FROM product_reviews WHERE product_id = 'P005' AND review_text LIKE '%蓝牙%' OR review_text LIKE '%连接%' LIMIT 10;"}}> | streaming
DEBUG | strands.event_loop.event_loop | tool_use=<{'toolUseId': '019875e9bd4d348fef3f02b0e39fa2e7', 'name': 'db_query', 'input': {'sql_stmt': 'DESCRIBE product_reviews;'}}> | streaming
DEBUG | strands.event_loop.event_loop | tool_use=<{'toolUseId': '019875e9f2810478947d1540aa27820a', 'name': 'db_query', 'input': {'sql_stmt': "SELECT review_id, product_id, customer_id, product_name, review_content, review_date FROM product_reviews WHERE product_id = 'P005' AND (review_content LIKE '%蓝牙%' OR review_content LIKE '%连接%') LIMIT 10;"}}> | streaming


In [64]:
res

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '目前我无法直接访问数据库来查询商品P005的蓝牙连接问题的评论信息，可能是因为接口问题或权限限制。  \n\n您可以尝试以下方法获取相关内容：  \n1. **检查商品页面**：通常商品详情页会展示用户的评论和反馈，尤其是关于蓝牙连接的问题。  \n2. **联系客服**：如果您需要更详细的帮助，可以直接联系平台客服，提供商品编号（P005）和具体问题（蓝牙连接问题），他们会为您提供更精准的解决方案。  \n\n如果您有其他查询需求或需要帮助的地方，请随时告诉我！'}]}, metrics=EventLoopMetrics(cycle_count=5, tool_metrics={'universal_search': ToolMetrics(tool={'toolUseId': '019875e7ecc60b69038c069c19c92131', 'name': 'universal_search', 'input': {'table_name': 'product_reviews', 'query': '蓝牙连接问题', 'filter_object': '{"product_id":{"$eq":"P005"}}', 'limit': '10'}}, call_count=2, success_count=0, error_count=2, total_time=2.2803778648376465)}, cycle_durations=[9.360470056533813, 5.802194356918335, 12.48629117012024], traces=[<strands.telemetry.metrics.Trace object at 0x7f38314af0d0>, <strands.telemetry.metrics.Trace object at 0x7f3830ba07c0>, <strands.telemetry.metrics.Trace object at 0x7f383054e080>, <strands.telemetry.metrics.Trace

从结果可以看到，他使用了universal_search 这个tool，参数是：

```
{'table_name': 'product_reviews', 'query': '蓝牙连接问题', 'filter_object': '{"product_id":{"$eq":"P005"}}', 'limit': '10'}}
```

我们可以直接用这个参数，调用TiDB的API：

In [74]:
pr_table.search("蓝牙连接问题").filter({"product_id":{"$eq":"P005"}}).limit(10).to_list()

[{'review_id': 'R004',
  'order_id': 'O003',
  'product_id': 'P005',
  'customer_id': 'C003',
  'rating': 2,
  'product_name': '品牌:TechZen 类型:蓝牙耳机 商品名:SoundBuds Pro 2',
  'review_content': '耳机连接不稳定，经常断连，音质也不如宣传的那么好，准备退货',
  'review_content_vec': array([-0.01396188,  0.03689145, -0.0274957 , ...,  0.02854494,
          0.00186262, -0.01442363], dtype=float32),
  'review_date': '2024-01-25 16:20:00',
  'is_verified_purchase': True,
  '_distance': 0.7440965016060754,
  '_score': 0.25590349839392457},
 {'review_id': 'R006',
  'order_id': 'O010',
  'product_id': 'P005',
  'customer_id': 'C009',
  'rating': 5,
  'product_name': '品牌:TechZen 类型:蓝牙耳机 商品名:SoundBuds Pro 2',
  'review_content': 'SoundBuds音质很好，佩戴舒适，降噪效果不错，非常满意',
  'review_content_vec': array([-0.03471835,  0.0601946 , -0.0186788 , ..., -0.0157928 ,
          0.00084577,  0.00078907], dtype=float32),
  'review_date': '2024-02-10 14:20:00',
  'is_verified_purchase': True,
  '_distance': 0.933538001830638,
  '_score': 0.06646199816936